# RT-DETR ShuffleNet Training

Training RT-DETR with ShuffleNetV2-Small backbone for WBC Classification on Raabin-WBC dataset.

## Model Details
- **Backbone**: ShuffleNetV2-Small
- **Training**: From scratch (no pretrained weights)
- **Dataset**: Raabin-WBC with 5 cell types

## 1. Setup and Imports

In [1]:
# %pip install -U ultralytics torch torchvision pillow tqdm scikit-learn seaborn timm

In [1]:
%matplotlib inline

import os
import json
import random
import shutil
import yaml
import time
from datetime import datetime

import numpy as np
import torch
from tqdm import tqdm

from ultralytics import RTDETR

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.6.0+cu124
CUDA available: True


## 2. Configuration

In [2]:
# =============================================================================
# MODEL CONFIGURATION
# =============================================================================
MODEL_NAME = "RT-DETR-ShuffleNet"
BACKBONE = "ShuffleNetV2-Small"
IS_PRETRAINED = False  # Training from scratch

# =============================================================================
# BASE DIRECTORY
# =============================================================================
NOTEBOOK_DIR = os.getcwd()
BASE_DIR = os.path.join(NOTEBOOK_DIR, "output")

# Dataset path
DATA_ROOT = r"C:\D drive\mydata\MSML\DataSets\Raabin_datsets_withlabels"

# Custom model YAML path
MODEL_YAML_PATH = os.path.join(NOTEBOOK_DIR, "rtdetr_shufflenetv2.yaml")

print(f"Notebook directory: {NOTEBOOK_DIR}")
print(f"Base directory: {BASE_DIR}")
print(f"Data root: {DATA_ROOT}")

# Verify YAML file exists
if os.path.exists(MODEL_YAML_PATH):
    print(f"Found model YAML: {MODEL_YAML_PATH}")
else:
    print(f"WARNING: Model YAML not found at: {MODEL_YAML_PATH}")

# =============================================================================
# SAMPLING CONFIGURATION
# =============================================================================
SAMPLES_PER_CLASS = 100  # Set to None for full dataset

# Data paths
IMAGES_DIR = os.path.join(DATA_ROOT, "Train", "images")
LABELS_DIR = os.path.join(DATA_ROOT, "Train", "labels")

# Output directories
os.makedirs(BASE_DIR, exist_ok=True)
MODEL_DIR = os.path.join(BASE_DIR, "models")
RESULTS_DIR = os.path.join(BASE_DIR, "results")
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

# Device configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Class definitions
CLASSES = {
    "Basophil": 0,
    "Eosinophil": 1,
    "Lymphocyte": 2,
    "Monocyte": 3,
    "Neutrophil": 4
}
ID2LABEL = {v: k for k, v in CLASSES.items()}
NUM_CLASSES = len(CLASSES)

print(f"\nUsing device: {DEVICE}")
print(f"Samples per class: {SAMPLES_PER_CLASS if SAMPLES_PER_CLASS else 'ALL'}")
print(f"\nModel: {MODEL_NAME} ({BACKBONE})")
print(f"Training mode: {'Pretrained' if IS_PRETRAINED else 'From scratch'}")

Notebook directory: C:\D drive\mydata\MSML\GitHub\RT-DETR-Based-Explainable-CAD-System-for-Automated-Detection-and-Classification-of-White-Blood-Cells
Base directory: C:\D drive\mydata\MSML\GitHub\RT-DETR-Based-Explainable-CAD-System-for-Automated-Detection-and-Classification-of-White-Blood-Cells\output
Data root: C:\D drive\mydata\MSML\DataSets\Raabin_datsets_withlabels
Found model YAML: C:\D drive\mydata\MSML\GitHub\RT-DETR-Based-Explainable-CAD-System-for-Automated-Detection-and-Classification-of-White-Blood-Cells\rtdetr_shufflenetv2.yaml

Using device: cuda
Samples per class: 100

Model: RT-DETR-ShuffleNet (ShuffleNetV2-Small)
Training mode: From scratch


## 3. Training Hyperparameters

In [3]:
# =============================================================================
# TRAINING HYPERPARAMETERS (FROM SCRATCH CONFIG)
# =============================================================================
# Training from scratch needs more epochs and lower learning rate

TRAINING_CONFIG = {
    "epochs": 20,           
    "imgsz": 640,
    "batch": 8,
    "lr0": 0.0001,          # Low LR for stability
    "lrf": 0.01,
    "momentum": 0.937,
    "weight_decay": 0.0005,
    "workers": 8,
    "patience": 20,
    "cos_lr": True,
    "warmup_epochs": 3,     # Gradual LR ramp-up for stability
    "warmup_momentum": 0.8,
    "warmup_bias_lr": 0.01,
}

print("Training Configuration:")
print("="*60)
for k, v in TRAINING_CONFIG.items():
    print(f"  {k}: {v}")

Training Configuration:
  epochs: 20
  imgsz: 640
  batch: 8
  lr0: 0.0001
  lrf: 0.01
  momentum: 0.937
  weight_decay: 0.0005
  workers: 8
  patience: 20
  cos_lr: True
  warmup_epochs: 3
  warmup_momentum: 0.8
  warmup_bias_lr: 0.01


## 4. Data Preparation

In [4]:
def create_training_subset(data_root, base_dir, classes, samples_per_class=None, random_seed=42):
    """
    Create a training subset with ZERO image duplication.
    - Creates train.txt/val.txt pointing to original images
    - Corrects class IDs in original label files (fixes the dataset)
    - No image files created in output folder
    """
    if random_seed is not None:
        random.seed(random_seed)
    
    # Define paths
    subset_dir = os.path.join(base_dir, "data_subset")
    
    # Source paths
    src_train_images = os.path.join(data_root, "Train", "images")
    src_train_labels = os.path.join(data_root, "Train", "labels")
    src_val_images = os.path.join(data_root, "val", "images")
    src_val_labels = os.path.join(data_root, "val", "labels")
    
    # Clean up existing subset directory
    if os.path.exists(subset_dir):
        shutil.rmtree(subset_dir)
    os.makedirs(subset_dir, exist_ok=True)
    
    # Lists to store image paths for txt files
    train_image_paths = []
    val_image_paths = []
    labels_corrected = 0
    
    total_train = 0
    total_val = 0
    
    for cls_name, cls_id in classes.items():
        # --- Training data ---
        src_cls_images = os.path.join(src_train_images, cls_name)
        src_cls_labels = os.path.join(src_train_labels, cls_name)
        
        if os.path.exists(src_cls_images):
            image_files = [f for f in os.listdir(src_cls_images) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            if samples_per_class is not None and len(image_files) > samples_per_class:
                image_files = random.sample(image_files, samples_per_class)
            
            for img_file in image_files:
                base_name = os.path.splitext(img_file)[0]
                
                # Store path to ORIGINAL image
                original_img_path = os.path.join(src_cls_images, img_file)
                train_image_paths.append(original_img_path)
                
                # Correct the ORIGINAL label file (fix class ID)
                label_file = base_name + ".txt"
                label_path = os.path.join(src_cls_labels, label_file)
                
                if os.path.exists(label_path):
                    with open(label_path, 'r') as f:
                        lines = f.readlines()
                    
                    new_lines = []
                    needs_correction = False
                    for line in lines:
                        parts = line.strip().split()
                        if len(parts) > 1:
                            if parts[0] != str(cls_id):
                                needs_correction = True
                                parts[0] = str(cls_id)
                            new_lines.append(' '.join(parts) + '\n')
                    
                    if needs_correction:
                        with open(label_path, 'w') as f:
                            f.writelines(new_lines)
                        labels_corrected += 1
            
            total_train += len(image_files)
            print(f"  {cls_name} (class {cls_id}): {len(image_files)} training images")
        
        # --- Validation data ---
        src_cls_val_images = os.path.join(src_val_images, cls_name)
        src_cls_val_labels = os.path.join(src_val_labels, cls_name)
        
        if os.path.exists(src_cls_val_images):
            val_files = [f for f in os.listdir(src_cls_val_images) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            if samples_per_class is not None:
                val_sample_size = min(len(val_files), max(20, samples_per_class // 5))
                if len(val_files) > val_sample_size:
                    val_files = random.sample(val_files, val_sample_size)
            
            for img_file in val_files:
                base_name = os.path.splitext(img_file)[0]
                
                # Store path to ORIGINAL image
                original_img_path = os.path.join(src_cls_val_images, img_file)
                val_image_paths.append(original_img_path)
                
                # Correct the ORIGINAL label file
                label_file = base_name + ".txt"
                label_path = os.path.join(src_cls_val_labels, label_file)
                
                if os.path.exists(label_path):
                    with open(label_path, 'r') as f:
                        lines = f.readlines()
                    
                    new_lines = []
                    needs_correction = False
                    for line in lines:
                        parts = line.strip().split()
                        if len(parts) > 1:
                            if parts[0] != str(cls_id):
                                needs_correction = True
                                parts[0] = str(cls_id)
                            new_lines.append(' '.join(parts) + '\n')
                    
                    if needs_correction:
                        with open(label_path, 'w') as f:
                            f.writelines(new_lines)
                        labels_corrected += 1
            
            total_val += len(val_files)
    
    # Write train.txt with paths to original images
    train_txt_path = os.path.join(subset_dir, "train.txt")
    with open(train_txt_path, 'w') as f:
        for img_path in train_image_paths:
            f.write(img_path + '\n')
    
    # Write val.txt with paths to original images
    val_txt_path = os.path.join(subset_dir, "val.txt")
    with open(val_txt_path, 'w') as f:
        for img_path in val_image_paths:
            f.write(img_path + '\n')
    
    # Create data.yaml pointing to txt files
    data_yaml_path = os.path.join(subset_dir, "data.yaml")
    data_config = {
        'path': data_root,  # Base path for label lookup
        'train': train_txt_path,  # Absolute path to train.txt
        'val': val_txt_path,  # Absolute path to val.txt
        'nc': len(classes),
        'names': {v: k for k, v in classes.items()}
    }
    
    with open(data_yaml_path, 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)
    
    print(f"\nSubset created:")
    print(f"  Total training images: {total_train}")
    print(f"  Total validation images: {total_val}")
    print(f"  Labels corrected: {labels_corrected}")
    print(f"  Data config: {data_yaml_path}")
    print(f"  NO image files copied - using original dataset directly!")
    
    return data_yaml_path

In [5]:
# Create training subset
print(f"Creating training subset with {SAMPLES_PER_CLASS if SAMPLES_PER_CLASS else 'ALL'} images per class...\n")
DATA_YAML = create_training_subset(
    DATA_ROOT, 
    BASE_DIR, 
    CLASSES, 
    samples_per_class=SAMPLES_PER_CLASS,
    random_seed=42
)

Creating training subset with 100 images per class...

  Basophil (class 0): 100 training images
  Eosinophil (class 1): 100 training images
  Lymphocyte (class 2): 100 training images
  Monocyte (class 3): 100 training images
  Neutrophil (class 4): 100 training images

Subset created:
  Total training images: 500
  Total validation images: 100
  Labels corrected: 0
  Data config: C:\D drive\mydata\MSML\GitHub\RT-DETR-Based-Explainable-CAD-System-for-Automated-Detection-and-Classification-of-White-Blood-Cells\output\data_subset\data.yaml
  NO image files copied - using original dataset directly!


## 5. Training

In [6]:
def train_model(model_yaml_path, model_name, data_yaml, training_config, base_dir):
    """
    Train the RT-DETR model and return training results.
    Clears previous training runs for this model before starting.
    """
    print(f"\n{'='*60}")
    print(f"Training: {model_name}")
    print(f"{'='*60}")

    # Project directory
    project_dir = os.path.join(base_dir, "training_runs")
    os.makedirs(project_dir, exist_ok=True)

    # Clear previous training runs for this model
    for folder in os.listdir(project_dir):
        if folder.startswith(model_name):
            old_run_path = os.path.join(project_dir, folder)
            print(f"Removing previous run: {folder}")
            shutil.rmtree(old_run_path)

    # Load model from YAML
    model = RTDETR(model_yaml_path)

    run_name = f"{model_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    # Record start time
    start_time = time.time()
    
    # Train
    results = model.train(
        data=data_yaml,
        epochs=training_config["epochs"],
        imgsz=training_config["imgsz"],
        batch=training_config["batch"],
        lr0=training_config["lr0"],
        lrf=training_config["lrf"],
        momentum=training_config["momentum"],
        weight_decay=training_config["weight_decay"],
        workers=training_config["workers"],
        patience=training_config["patience"],
        cos_lr=training_config["cos_lr"],
        warmup_epochs=training_config.get("warmup_epochs", 3),
        warmup_momentum=training_config.get("warmup_momentum", 0.8),
        warmup_bias_lr=training_config.get("warmup_bias_lr", 0.1),
        project=project_dir,
        name=run_name,
        save=True,
        plots=True,
        verbose=True,
    )
    
    training_time = time.time() - start_time
    
    # Get best model path
    best_model_path = os.path.join(project_dir, run_name, "weights", "best.pt")
    
    return {
        "model_name": model_name,
        "best_model_path": best_model_path,
        "training_time": training_time,
        "run_dir": os.path.join(project_dir, run_name),
        "results": results,
    }

In [7]:
# Train the model
training_result = train_model(
    MODEL_YAML_PATH,
    MODEL_NAME,
    DATA_YAML,
    TRAINING_CONFIG,
    BASE_DIR
)

print(f"\nTraining completed in {training_result['training_time']:.1f}s")
print(f"Best model saved to: {training_result['best_model_path']}")


Training: RT-DETR-ShuffleNet
Removing previous run: RT-DETR-ShuffleNet_20260131_102555
New https://pypi.org/project/ultralytics/8.4.9 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.8  Python-3.12.10 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=C:\D drive\mydata\MSML\GitHub\RT-DETR-Based-Explainable-CAD-System-for-Automated-Detection-and-Classification-of-White-Blood-Cells\output\data_subset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=

C:\Users\sriva\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\autograd\graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:96.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/20      4.22G      1.856      5.348      1.686         11        640: 100% ━━━━━━━━━━━━ 63/63 2.9it/s 21.7s0.3ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 6.9it/s 1.0s0.2s
                   all        100        155   0.000157      0.191   0.000467   0.000127

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       2/20       4.3G      1.521     0.5375      1.153         19        640: 0% ──────────── 0/63  0.4s

C:\Users\sriva\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\autograd\graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:96.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       2/20      4.38G      1.552      0.638      1.386         10        640: 100% ━━━━━━━━━━━━ 63/63 3.2it/s 19.9s0.3ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 7.3it/s 1.0s0.2s
                   all        100        155   0.000205      0.134   0.000293   6.64e-05

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       3/20      4.22G      1.586     0.6002      1.356         18        640: 0% ──────────── 0/63  0.3s

...

      18/20      4.39G     0.4207      1.159     0.2966          5        640: 100% ━━━━━━━━━━━━ 63/63 3.3it/s 19.3s0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 7.3it/s 1.0s0.2s
                   all        100        155      0.475       0.44      0.389      0.291

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      19/20       4.3G     0.2563      1.132     0.1975         11        640: 0% ──────────── 0/63  0.3s

C:\Users\sriva\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\autograd\graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:96.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      19/20      4.35G     0.4395      1.132     0.3077          4        640: 100% ━━━━━━━━━━━━ 63/63 3.3it/s 19.2s0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 7.0it/s 1.0s0.2s
                   all        100        155      0.358      0.605      0.444      0.333

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      20/20      4.31G     0.3246      1.337     0.2021         13        640: 0% ──────────── 0/63  0.3s

C:\Users\sriva\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\autograd\graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:96.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      20/20      4.39G     0.4557      1.142     0.3077          4        640: 100% ━━━━━━━━━━━━ 63/63 3.3it/s 19.2s0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 7.2it/s 1.0s0.2s
                   all        100        155      0.356      0.601      0.434      0.319

20 epochs completed in 0.130 hours.
Optimizer stripped from C:\D drive\mydata\MSML\GitHub\RT-DETR-Based-Explainable-CAD-System-for-Automated-Detection-and-Classification-of-White-Blood-Cells\output\training_runs\RT-DETR-ShuffleNet_20260131_103958\weights\last.pt, 38.7MB
Optimizer stripped from C:\D drive\mydata\MSML\GitHub\RT-DETR-Based-Explainable-CAD-System-for-Automated-Detection-and-Classification-of-White-Blood-Cells\output\training_runs\RT-DETR-ShuffleNet_20260131_103958\weights\best.pt, 38.7MB

Validating C:\D drive\mydata\MSML\GitHub\RT-DETR-Based-Explainable-CAD-System-for-Automated-Detection-and-Classification-of-White-Blood-Cells\output\trai

## 6. Evaluation

In [8]:
def evaluate_model(model_path, images_dir, classes, id2label, 
                   conf_thresh=0.1, eval_per_class=100, random_seed=123):
    """
    Evaluate the trained model on the dataset.
    """
    model = RTDETR(model_path)
    
    random.seed(random_seed)
    
    y_true = []
    y_pred = []
    inference_times = []
    
    for gt_class, gt_id in classes.items():
        cls_dir = os.path.join(images_dir, gt_class)
        files = [f for f in os.listdir(cls_dir) if f.lower().endswith(".jpg")]
        
        if len(files) > eval_per_class:
            files = random.sample(files, eval_per_class)
        
        for fname in tqdm(files, desc=f"Evaluating {gt_class}", leave=False):
            img_path = os.path.join(cls_dir, fname)
            
            start = time.time()
            results = model(img_path, conf=conf_thresh, verbose=False)[0]
            inference_times.append(time.time() - start)
            
            y_true.append(gt_id)
            
            if len(results.boxes) == 0:
                y_pred.append(-1)
            else:
                best_idx = results.boxes.conf.argmax()
                y_pred.append(int(results.boxes.cls[best_idx].cpu().item()))
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate metrics
    valid = y_pred != -1
    valid_count = np.sum(valid)
    
    if valid_count > 0:
        accuracy = accuracy_score(y_true[valid], y_pred[valid])
        cm = confusion_matrix(y_true[valid], y_pred[valid], labels=list(range(len(classes))))
        report = classification_report(
            y_true[valid], y_pred[valid],
            target_names=list(classes.keys()),
            labels=list(range(len(classes))),
            zero_division=0,
            output_dict=True
        )
    else:
        accuracy = 0.0
        cm = None
        report = None
    
    return {
        "accuracy": accuracy,
        "no_prediction_count": len(y_true) - valid_count,
        "total_samples": len(y_true),
        "confusion_matrix": cm.tolist() if cm is not None else None,
        "classification_report": report,
        "avg_inference_time": np.mean(inference_times),
        "y_true": y_true.tolist(),
        "y_pred": y_pred.tolist(),
    }

In [9]:
# Evaluate the model
CONF_THRESH = 0.1
EVAL_PER_CLASS = 100

print(f"Evaluating: {MODEL_NAME}")
evaluation_result = evaluate_model(
    model_path=training_result["best_model_path"],
    images_dir=IMAGES_DIR,
    classes=CLASSES,
    id2label=ID2LABEL,
    conf_thresh=CONF_THRESH,
    eval_per_class=EVAL_PER_CLASS,
)

print(f"\nResults:")
print(f"  Accuracy: {evaluation_result['accuracy']:.4f}")
print(f"  Avg inference time: {evaluation_result['avg_inference_time']*1000:.2f}ms")
print(f"  No predictions: {evaluation_result['no_prediction_count']}/{evaluation_result['total_samples']}")

Evaluating: RT-DETR-ShuffleNet



Results:
  Accuracy: 0.6640
  Avg inference time: 31.11ms
  No predictions: 0/500


In [10]:
# Print classification report
if evaluation_result["classification_report"] is not None:
    y_true = np.array(evaluation_result["y_true"])
    y_pred = np.array(evaluation_result["y_pred"])
    valid = y_pred != -1
    
    print(f"\n--- {MODEL_NAME} Classification Report ---")
    print(classification_report(
        y_true[valid],
        y_pred[valid],
        target_names=list(CLASSES.keys()),
        labels=list(range(NUM_CLASSES)),
        zero_division=0
    ))


--- RT-DETR-ShuffleNet Classification Report ---
              precision    recall  f1-score   support

    Basophil       0.90      0.86      0.88       100
  Eosinophil       0.74      0.14      0.24       100
  Lymphocyte       0.58      0.93      0.71       100
    Monocyte       0.69      0.51      0.59       100
  Neutrophil       0.59      0.88      0.70       100

    accuracy                           0.66       500
   macro avg       0.70      0.66      0.62       500
weighted avg       0.70      0.66      0.62       500



## 7. Save Results to Disk

In [11]:
# Prepare results for saving (convert numpy types to native Python)
def convert_to_native(obj):
    """Convert numpy types to native Python types for JSON serialization."""
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.int64, np.int32, np.int16, np.int8)):
        return int(obj)
    elif isinstance(obj, (np.float64, np.float32, np.float16)):
        return float(obj)
    elif isinstance(obj, dict):
        return {k: convert_to_native(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native(i) for i in obj]
    return obj

results_to_save = {
    "model_name": MODEL_NAME,
    "backbone": BACKBONE,
    "is_pretrained": IS_PRETRAINED,
    "best_model_path": training_result["best_model_path"],
    "run_dir": training_result["run_dir"],
    "training_time_s": float(training_result["training_time"]),
    "training_config": TRAINING_CONFIG,
    "accuracy": float(evaluation_result["accuracy"]),
    "avg_inference_time_ms": float(evaluation_result["avg_inference_time"]) * 1000,
    "no_prediction_count": int(evaluation_result["no_prediction_count"]),
    "total_samples": int(evaluation_result["total_samples"]),
    "confusion_matrix": convert_to_native(evaluation_result["confusion_matrix"]),
    "classification_report": convert_to_native(evaluation_result["classification_report"]),
    "y_true": convert_to_native(evaluation_result["y_true"]),
    "y_pred": convert_to_native(evaluation_result["y_pred"]),
    "classes": CLASSES,
    "timestamp": datetime.now().isoformat(),
}

# Save to JSON
results_file = os.path.join(RESULTS_DIR, f"{MODEL_NAME}_results.json")
with open(results_file, 'w') as f:
    json.dump(results_to_save, f, indent=2)

print(f"Results saved to: {results_file}")

Results saved to: C:\D drive\mydata\MSML\GitHub\RT-DETR-Based-Explainable-CAD-System-for-Automated-Detection-and-Classification-of-White-Blood-Cells\output\results\RT-DETR-ShuffleNet_results.json


In [12]:
# Summary
print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)
print(f"Model: {MODEL_NAME} ({BACKBONE})")
print(f"Accuracy: {evaluation_result['accuracy']:.4f}")
print(f"Inference Time: {evaluation_result['avg_inference_time']*1000:.2f}ms")
print(f"Training Time: {training_result['training_time']:.1f}s")
print(f"\nBest model: {training_result['best_model_path']}")
print(f"Results JSON: {results_file}")
print("="*60)


TRAINING COMPLETE
Model: RT-DETR-ShuffleNet (ShuffleNetV2-Small)
Accuracy: 0.6640
Inference Time: 31.11ms
Training Time: 548.6s

Best model: C:\D drive\mydata\MSML\GitHub\RT-DETR-Based-Explainable-CAD-System-for-Automated-Detection-and-Classification-of-White-Blood-Cells\output\training_runs\RT-DETR-ShuffleNet_20260131_103958\weights\best.pt
Results JSON: C:\D drive\mydata\MSML\GitHub\RT-DETR-Based-Explainable-CAD-System-for-Automated-Detection-and-Classification-of-White-Blood-Cells\output\results\RT-DETR-ShuffleNet_results.json
